In [38]:
!pip3 install yfinance==0.2.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 3.2 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.11
    Uninstalling yfinance-0.2.11:
      Successfully uninstalled yfinance-0.2.11


In [39]:
import pandas as pd
import yfinance as yf

In [40]:
TRANSAC_INFO = ['ticker', 'price', 'quantity']
INFO_TYPE = [str, float, int]
LENTH_INFO = len(TRANSAC_INFO)

In [95]:
class Agent:

    def __init__ (self, name="The agent"):
        
        '''
        create a agent with given name having $1000
        
        Arguments
        name [str]: name of the agent (default: The agent)
        '''
        self.credit = 1000
        self.holdings = pd.DataFrame(index=["avg_price", "num"], dtype="float")
        self.name = name

        print(f"{self.name} is created.")

    def check(self):
    
        '''
        print current state(credit and holdings) of the agent
        '''
        print(f"credit : {self.credit}")
        print(f"holding : {self.df_to_list(self.holdings)}")
    
    def buy(self, buy_inform : list):
        
        '''
        buy stocks according to buy_inform

        Arguments
        buy_inform [list]: information of the stock to buy [ticker : str, price : float|int, quantity : int]
        '''
        # check argement validation
        if not isinstance(buy_inform, list):
            raise TypeError('buy_inform must be a list')
        if len(buy_inform) != LENTH_INFO:
            raise ValueError(f'Unexpected length of buy_inform\n\
                            buy_inform must have {LENTH_INFO} elements, {TRANSAC_INFO}')
        if isinstance(buy_inform[1], int): buy_inform[1] = float(buy_inform[1])
        for i in range(LENTH_INFO):
            if not isinstance(buy_inform[i], INFO_TYPE[i]):
                raise TypeError(f'{TRANSAC_INFO[i]} must be a {INFO_TYPE[i]}')

        ticker, price, quantity = buy_inform
        
        # check validity of ticker
        if not self.is_valid_ticker(ticker):
            raise ValueError(f'{ticker} is not valid')
        
        # upper case
        ticker = ticker.upper()

        if price*quantity > self.credit:
            print(f"{self.name} can't buy.")
            return
        
        if ticker in self.holdings.columns:
            ticker_holdings = self.holdings[ticker]
            prev_avg_price = ticker_holdings["avg_price"]
            prev_num = ticker_holdings["num"]
            ticker_holdings["avg_price"] = (prev_avg_price*prev_num + price*quantity)/(prev_num + quantity)
            ticker_holdings["num"] += quantity
        else:
            self.holdings[ticker] = [price, quantity]

        self.credit -= price*quantity

        print(f"{self.name} buys {int(quantity)} {ticker} for {price}.")
        return

    def sell(self, sell_inform : list):
        
        '''
        sell stocks according to sell_inform

        Arguments
        sell_inform [list]: information of the stock to sell [ticker : str, price : float|int, quantity : int]
        '''
        # check argement validation
        if not isinstance(sell_inform, list):
            raise TypeError('sell_inform must be a list')
        if len(sell_inform) != LENTH_INFO:
            raise ValueError(f'Unexpected length of sell_inform\n\
                            buy_inform must have {LENTH_INFO} elements, {TRANSAC_INFO}')
        if isinstance(sell_inform[1], int): sell_inform[1] = float(sell_inform[1])
        for i in range(LENTH_INFO):
            if not isinstance(sell_inform[i], INFO_TYPE[i]):
                raise TypeError(f'{TRANSAC_INFO[i]} must be a {INFO_TYPE[i]}')

        ticker, price, quantity = sell_inform
        
        # check validity of ticker
        if not self.is_valid_ticker(ticker):
            raise ValueError(f'{ticker} is not valid')
        
        # upper case
        ticker = ticker.upper()

        if not ticker in self.holdings.columns or quantity > self.holdings[ticker]["num"]:
            print(f"{self.name} can't sell.")
            return
        
        ticker_holdings = self.holdings[ticker]
        ticker_holdings["num"] -= quantity
        self.credit += price*quantity

        print(f"{self.name} sells {int(quantity)} {ticker} for {price}.")
        return

    def df_to_list(self, df : pd.DataFrame):
        '''
        convert dataframe object to list

        Arguments
        df [pd.DataFrame]: DataFrame of holdings (index: avg_price, num; columns: tickers)
        '''
        return [ [ticker] + df[ticker].tolist() for ticker in df.columns]

    def is_valid_ticker(self, ticker : str):
        '''
        check validity of the ticker
        '''
        yf_ticker = yf.Ticker(ticker)
        if yf_ticker.info == None: 
            print(f"{ticker} is not Valid.")
            return False
        return True

In [96]:
FBA_agent = Agent()

The agent is created.


In [97]:
FBA_agent.check()

credit : 1000
holding : []


In [98]:
FBA_agent.buy(['AAPL', 135, 5])

The agent buys 5 AAPL for 135.0.


In [99]:
FBA_agent.check()

credit : 325.0
holding : [['AAPL', 135.0, 5.0]]


In [100]:
FBA_agent.buy(['GOOGL', 2300, 1])

The agent can't buy.


In [101]:
FBA_agent.sell(['AAPL', 140, 3])

The agent sells 3 AAPL for 140.0.


In [102]:
FBA_agent.check()

credit : 745.0
holding : [['AAPL', 135.0, 2.0]]


In [103]:
FBA_agent.buy(['GOOGL', 210, 3])

The agent buys 3 GOOGL for 210.0.


In [104]:
FBA_agent.check()

credit : 115.0
holding : [['AAPL', 135.0, 2.0], ['GOOGL', 210.0, 3.0]]


In [105]:
FBA_agent.sell(['AAPL', 140, 3])

The agent can't sell.


In [106]:
FBA_agent.check()

credit : 115.0
holding : [['AAPL', 135.0, 2.0], ['GOOGL', 210.0, 3.0]]


In [107]:
FBA_agent.sell(['TSLA', 190, 3])

The agent can't sell.


In [108]:
FBA_agent.check()

credit : 115.0
holding : [['AAPL', 135.0, 2.0], ['GOOGL', 210.0, 3.0]]


In [109]:
FBA_agent.buy(['SYLEE', 210, 3])

SYLEE: No data found, symbol may be delisted
SYLEE is not Valid.


ValueError: ignored